In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import os

# Function to handle chatbot summarization
from langchain_community.document_loaders import JSONLoader
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
store = {}

In [3]:
def parentChatbotSummarize(filepath: str, session_id: str, promptQuestion: str):
    try:
        llm = ChatOpenAI(model='gpt-4o')

        loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
        splits = text_splitter.split_documents(docs)
        vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=OpenAIEmbeddings())
        retriever = vectorstore.as_retriever(search_type='similarity')

        contextualize_q_system_prompt = (
            "Given a chat history and the latest user question "
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )

        contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )

        history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
        system_prompt = (
            "You are a nurse updating a parent on the status of their child in the NICU."
            "Use the following retrieved context to provide relevant information to the parent. "
            "Speak in a conversational tone, and keep the summary concise."
            "Do not diagnose."
            "\n\n{context}"
        )

        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
        question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
        rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

        conversational_rag_chain = RunnableWithMessageHistory(
            rag_chain,
            get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer",
        )
        response = conversational_rag_chain.invoke(
            {"input": promptQuestion},
            config={
                "configurable": {"session_id": session_id}
            },  # constructs a key "abc123" in `store`.
            
        )

        return response['answer']
    except Exception as e:
        return f"Error: {str(e)}"

In [4]:
# Function triggered by GUI button
def run_chatbot():
    filepath = file_entry.get()
    session_id = session_entry.get()
    prompt = prompt_entry.get()

    if not os.path.exists(filepath):
        messagebox.showerror("Error", "The selected file does not exist.")
        return

    if not session_id:
        messagebox.showerror("Error", "Please enter a valid session ID.")
        return

    if not prompt:
        messagebox.showerror("Error", "Please enter a prompt.")
        return

    result = parentChatbotSummarize(filepath, session_id, prompt)
    chat_history_text.configure(state='normal')
    chat_history_text.insert(tk.END, f"You: {prompt}\n")
    chat_history_text.insert(tk.END, f"Bot: {result}\n\n")
    chat_history_text.configure(state='disabled')
    prompt_entry.delete(0, tk.END)

In [5]:
# GUI Setup using Tkinter
root = tk.Tk()
root.title("Chatbot GUI for JSON Summarization")
root.geometry("600x500")

# Filepath Entry
file_label = tk.Label(root, text="Select JSON File:")
file_label.pack(pady=5)
file_entry = tk.Entry(root, width=50)
file_entry.pack(pady=5)
file_button = tk.Button(root, text="Browse", command=lambda: file_entry.insert(0, filedialog.askopenfilename(filetypes=[("JSON Files", "*.json")])))
file_button.pack(pady=5)

# Session ID Entry
session_label = tk.Label(root, text="Enter Session ID:")
session_label.pack(pady=5)
session_entry = tk.Entry(root, width=50)
session_entry.pack(pady=5)

# Prompt Entry
prompt_label = tk.Label(root, text="Enter Prompt:")
prompt_label.pack(pady=5)
prompt_entry = tk.Entry(root, width=50)
prompt_entry.pack(pady=5)

# Chat History Display
chat_history_text = scrolledtext.ScrolledText(root, width=70, height=15, state='disabled', wrap='word')
chat_history_text.pack(pady=10)

# Run Button
run_button = tk.Button(root, text="Run Chatbot", command=run_chatbot)
run_button.pack(pady=20)

In [6]:
# Mainloop to run the GUI
root.mainloop()